# Import packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import expected_returns
from datetime import datetime
from pandas.tseries.offsets import BDay

In [3]:
import time
import pickle

# 1. Read Input Data

In [4]:
df_price = pd.read_csv("sp500_price_19960101_20221021.csv")

In [5]:
df_price.shape

(6217423, 45)

In [6]:
df_price['adj_price'] = df_price['prccd'] / df_price['ajexdi']

In [7]:
df_price = df_price[["gvkey", "datadate", 'adj_price']]

In [8]:
len(df_price.gvkey.unique())

982

In [9]:
selected_stock = pd.read_csv("stock_selected.csv",index_col=0)

In [10]:
selected_stock=selected_stock[selected_stock.trade_date>='2018-03-01'].reset_index(drop=True)

In [11]:
selected_stock.shape

(3870, 3)

In [13]:
selected_stock.head()

,gvkey,predicted_return,trade_date
0,1678,0.013570,2018-03-01
1,4430,0.012420,2018-03-01
2,4503,0.002985,2018-03-01
3,6788,0.033034,2018-03-01
4,7912,0.012037,2018-03-01


# 2. Get trade date

In [14]:
print("Number of unique stocks selected: ", len(selected_stock.gvkey.unique()))

Number of unique stocks selected:  644


In [16]:
all_date=df_price.datadate.unique()

In [17]:
len(all_date)

6886

In [18]:
trade_date=selected_stock.trade_date.unique()

In [19]:
trade_date

array(['2018-03-01', '2018-06-01', '2018-09-01', '2018-12-01',
       '2019-03-01', '2019-06-01', '2019-09-01', '2019-12-01',
       '2020-03-01', '2020-06-01', '2020-09-01', '2020-12-01',
       '2021-03-01', '2021-06-01', '2021-09-01', '2021-12-01',
       '2022-03-01', '2022-06-01', '2022-09-01'], dtype=object)

In [20]:
print("Number of trade dates", len(trade_date))

Number of trade dates 19


# 3. Get daily 1 year return table in each 89 trade period

In [21]:
selected_stock.head()

,gvkey,predicted_return,trade_date
0,1678,0.013570,2018-03-01
1,4430,0.012420,2018-03-01
2,4503,0.002985,2018-03-01
3,6788,0.033034,2018-03-01
4,7912,0.012037,2018-03-01


In [22]:
# took about 9 minutes to run
start = time.time()
all_return_table={}
#all_predicted_return={}
all_stocks_info = {}
#for i in range(0,1):
for i in range(len(trade_date)):
    #match trading date
    index = selected_stock.trade_date==trade_date[i]
    #get the corresponding trade period's selected stocks' name
    stocks_name=selected_stock.gvkey[selected_stock.trade_date==trade_date[i]].values
    temp_info = selected_stock[selected_stock.trade_date==trade_date[i]]
    temp_info = temp_info.reset_index()
    del temp_info['index']
    all_stocks_info[trade_date[i]] = temp_info
    #get the corresponding trade period's selected stocks' predicted return
    asset_expected_return=selected_stock[index].predicted_return.values
    
    
    #determine the business date
    #print(convert_to_yyyymmdd)
    tradedate = pd.to_datetime(trade_date[i])
    ts = datetime(tradedate.year-1, tradedate.month, tradedate.day)
    bd = pd.tseries.offsets.BusinessDay(n =1) 
    new_timestamp = ts - bd 
    all_date = pd.to_datetime(all_date, format="%Y%m%d")
    get_date_index=(all_date<tradedate) & (all_date>new_timestamp)
    get_date=all_date[get_date_index]
    #get adjusted price table
    return_table=pd.DataFrame()
    for m in range(len(stocks_name)):
        #get stocks's name
        index_tic=(df_price.gvkey==stocks_name[m])
        #get this stock's all historicall price from sp500_price
        sp500_temp=df_price[index_tic]
        merge_left_data_table = pd.DataFrame(get_date)
        merge_left_data_table.columns = ['datadate']
        #print(merge_left_data_table)
        sp500_temp.datadate = pd.to_datetime(sp500_temp.datadate, format="%Y%m%d")
        temp_price=merge_left_data_table.merge(sp500_temp, on=['datadate'], how='left')
        
        temp_price = temp_price.dropna()
        temp_price['daily_return']=temp_price.adj_price.pct_change()
        return_table=return_table.append(temp_price,ignore_index=True)
    all_return_table[trade_date[i]] = return_table
end = time.time()
print("Time consuming: ", (end-start)/60, " minutes")
    
    

Time consuming:  1.1987383445103963  minutes


In [23]:
check=all_stocks_info['2018-03-01'].gvkey.unique()

In [24]:
for i in check:
    if i not in all_return_table['2018-03-01'].gvkey.unique():
        print(i)

In [25]:
df_price.head()

,gvkey,datadate,adj_price
0,1045,19960102,37.5625
1,1045,19960103,38.4375
2,1045,19960104,37.6875
3,1045,19960105,37.3125
4,1045,19960108,36.7500


In [26]:
selected_stock[selected_stock.gvkey==34443]

,gvkey,predicted_return,trade_date
382,34443,0.024301,2021-03-01
393,34443,0.010315,2021-06-01
404,34443,0.016737,2021-09-01


In [27]:
df_price[df_price.gvkey==34443]

,gvkey,datadate,adj_price
4857154,34443,20190320,49.80
4857155,34443,20190321,48.98
4857156,34443,20190322,48.60
4857157,34443,20190325,49.15
4857158,34443,20190326,48.85
...,...,...,...
4858056,34443,20221017,45.26
4858057,34443,20221018,46.38
4858058,34443,20221019,45.13
4858059,34443,20221020,44.93


## Save to pickle

In [ ]:
with open('all_return_table.pickle', 'wb') as handle: 
    pickle.dump(all_return_table, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('all_stocks_info.pickle', 'wb') as handle:
    pickle.dump(all_stocks_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('all_return_table.pickle', 'rb') as handle:
    all_return_table = pickle.load(handle)

with open('all_stocks_info.pickle', 'rb') as handle:
    all_stocks_info = pickle.load(handle)


In [28]:
len(all_stocks_info)

19

In [29]:
all_return_table['2018-03-01']

,datadate,gvkey,adj_price,daily_return
0,2017-03-01,1678.0,52.95,NaN
1,2017-03-02,1678.0,51.92,-0.019452
2,2017-03-03,1678.0,52.09,0.003274
3,2017-03-06,1678.0,52.19,0.001920
4,2017-03-07,1678.0,51.28,-0.017436
...,...,...,...,...
55430,2018-02-22,260774.0,44.78,0.006745
55431,2018-02-23,260774.0,45.95,0.026128
55432,2018-02-26,260774.0,46.64,0.015016
55433,2018-02-27,260774.0,47.07,0.009220


In [30]:
len(all_return_table)

19

In [31]:
all_stocks_info['2018-03-01']

,gvkey,predicted_return,trade_date
0,1678,0.013570,2018-03-01
1,4430,0.012420,2018-03-01
2,4503,0.002985,2018-03-01
3,6788,0.033034,2018-03-01
4,7912,0.012037,2018-03-01
...,...,...,...
175,32580,0.046215,2018-03-01
176,65290,0.020053,2018-03-01
177,65640,0.045059,2018-03-01
178,110179,0.020612,2018-03-01


# 4. Potfolio Optimization using pypfopt

In [32]:
# took under 5 minutes to run
from pypfopt import objective_functions
stocks_weight_table = pd.DataFrame([])

for i in range(len(trade_date)):
    # get selected stocks information
    p1_alldata=(all_stocks_info[trade_date[i]])
    # sort it by tic
    p1_alldata=p1_alldata.sort_values('gvkey')
    p1_alldata = p1_alldata.reset_index()
    del p1_alldata['index']
    
    print(p1_alldata)
    # get selected stocks tic
    p1_stock = p1_alldata.gvkey
    
    # get predicted return from selected stocks
    p1_predicted_return=p1_alldata.pivot_table(index = 'trade_date',columns = 'gvkey', values = 'predicted_return')
    # use the predicted returns as the Expected returns to feed into the portfolio object
    

    # get the 1-year historical return
    p1_return_table=all_return_table[trade_date[i]]
    p1_return_table_pivot=p1_return_table.pivot_table(index = 'datadate',columns = 'gvkey', values = 'daily_return')
    print(p1_return_table)
    selected_stocks = list(set(p1_predicted_return.columns).intersection(p1_return_table_pivot.columns))
    # use the 1-year historical return table to calculate covariance matrix between selected stocks
    p1_predicted_return = p1_predicted_return.loc[:, selected_stocks]
    p1_return_table_pivot = p1_return_table_pivot.loc[:, selected_stocks]
    S = risk_models.sample_cov(p1_return_table_pivot)
    mu = p1_predicted_return.T.values
   # del S.index.name 
    # mean variance
    ef_mean = EfficientFrontier(mu, S,weight_bounds=(0, 0.05))
    # raw_weights_mean = ef_mean.max_sharpe()
    raw_weights_mean = ef_mean.nonconvex_objective(
        objective_functions.sharpe_ratio,
        objective_args = (ef_mean.expected_returns, ef_mean.cov_matrix),
        weights_sum_to_one = True
    )
    cleaned_weights_mean = ef_mean.clean_weights()
    #print(raw_weights_mean)
    #ef.portfolio_performance(verbose=True)
    #print("cleaned_weights_mean: ", cleaned_weights_mean)

    # minimum variance
    ef_min = EfficientFrontier([0]*len(mu), S,weight_bounds=(0, 0.05))
   # raw_weights_min = ef_min.max_sharpe()
    raw_weights_min = ef_min.nonconvex_objective(
        objective_functions.sharpe_ratio,
        objective_args = (ef_min.expected_returns, ef_min.cov_matrix),
        weights_sum_to_one = True
    )
    cleaned_weights_min = ef_min.clean_weights()
    #print(cleaned_weights_min)
    print("cleaned_weights_min: ", cleaned_weights_min)

    idx = np.isin(p1_alldata.gvkey, selected_stocks)
    p1_alldata["mean_weight"] = 0
    p1_alldata['mean_weight'][idx] = list(cleaned_weights_mean.values())
    p1_alldata["min_weight"] = 0
    p1_alldata['min_weight'][idx] = list(cleaned_weights_min.values())
    p1_alldata["equal_weight"] = 0
    p1_alldata['equal_weight'][idx] = np.ones(len(cleaned_weights_mean.values())) / len(cleaned_weights_mean.values())
    #ef.portfolio_performance(verbose=True)
    
    stocks_weight_table = stocks_weight_table.append(pd.DataFrame(p1_alldata), ignore_index=True)
    print(trade_date[i], ": Done")


      gvkey  predicted_return  trade_date
0      1230          0.026920  2018-03-01
1      1678          0.013570  2018-03-01
2      1722          0.035988  2018-03-01
3      2574          0.035439  2018-03-01
4      2751          0.028170  2018-03-01
..      ...               ...         ...
175  254338          0.033301  2018-03-01
176  260774          0.022298  2018-03-01
177  270281          0.027811  2018-03-01
178  287882          0.026155  2018-03-01
179  294524          0.026678  2018-03-01

[180 rows x 3 columns]
        datadate     gvkey  adj_price  daily_return
0     2017-03-01    1678.0      52.95           NaN
1     2017-03-02    1678.0      51.92     -0.019452
2     2017-03-03    1678.0      52.09      0.003274
3     2017-03-06    1678.0      52.19      0.001920
4     2017-03-07    1678.0      51.28     -0.017436
...          ...       ...        ...           ...
55430 2018-02-22  260774.0      44.78      0.006745
55431 2018-02-23  260774.0      45.95      0.026128
5543

cleaned_weights_min:  OrderedDict([(11264.0, 0.00532), (6669.0, 0.00532), (2574.0, 0.00532), (189459.0, 0.00532), (8215.0, 0.0053), (6682.0, 0.00532), (4640.0, 0.00532), (28192.0, 0.00532), (3107.0, 0.00532), (11300.0, 0.00532), (20004.0, 0.00533), (29736.0, 0.00532), (10793.0, 0.00534), (10795.0, 0.00533), (26156.0, 0.00532), (61483.0, 0.00532), (1075.0, 0.00533), (30259.0, 0.00533), (15414.0, 0.00532), (9783.0, 0.00532), (253501.0, 0.00532), (1602.0, 0.00533), (25157.0, 0.00532), (28742.0, 0.00532), (162887.0, 0.00532), (3144.0, 0.00532), (23627.0, 0.00532), (183377.0, 0.00533), (9299.0, 0.00532), (152149.0, 0.00532), (12892.0, 0.00532), (110179.0, 0.00532), (12389.0, 0.00532), (65640.0, 0.00532), (163946.0, 0.00532), (186989.0, 0.00533), (4093.0, 0.00533), (5234.0, 0.00532), (10867.0, 0.00534), (177267.0, 0.00532), (9846.0, 0.00531), (8823.0, 0.00533), (64630.0, 0.00532), (63099.0, 0.00533), (6781.0, 0.00532), (160893.0, 0.00532), (2176.0, 0.00533), (61574.0, 0.00532), (11399.0, 0.0

cleaned_weights_min:  OrderedDict([(11264.0, 0.00536), (4611.0, 0.00536), (7171.0, 0.00536), (2574.0, 0.00536), (31774.0, 0.00536), (25119.0, 0.00536), (4640.0, 0.00536), (28192.0, 0.00536), (3107.0, 0.00536), (20004.0, 0.00536), (10793.0, 0.00536), (61483.0, 0.00536), (9783.0, 0.00536), (29241.0, 0.00536), (1602.0, 0.00536), (4674.0, 0.00536), (7750.0, 0.00536), (28742.0, 0.00536), (8264.0, 0.00535), (23627.0, 0.00536), (8272.0, 0.00536), (183377.0, 0.00536), (9299.0, 0.00536), (152149.0, 0.00536), (126554.0, 0.00536), (110179.0, 0.00536), (142953.0, 0.00536), (2154.0, 0.00536), (34410.0, 0.00536), (29804.0, 0.00536), (186989.0, 0.00536), (5234.0, 0.00536), (10867.0, 0.00536), (9846.0, 0.00536), (8823.0, 0.00536), (64630.0, 0.00536), (18043.0, 0.00536), (63099.0, 0.00536), (61567.0, 0.00536), (4737.0, 0.00536), (5256.0, 0.00536), (287882.0, 0.00536), (20116.0, 0.00536), (145046.0, 0.00536), (61591.0, 0.00536), (316056.0, 0.00536), (25753.0, 0.00536), (5786.0, 0.00536), (9882.0, 0.0053

cleaned_weights_min:  OrderedDict([(10247.0, 0.00537), (160776.0, 0.00535), (9225.0, 0.00536), (100873.0, 0.00535), (6669.0, 0.00535), (2574.0, 0.00535), (189459.0, 0.00535), (8215.0, 0.00535), (6682.0, 0.00535), (31774.0, 0.00535), (18465.0, 0.00535), (3107.0, 0.00535), (20004.0, 0.00535), (10793.0, 0.00535), (162346.0, 0.00535), (61483.0, 0.00536), (30259.0, 0.00535), (9783.0, 0.00535), (187450.0, 0.00535), (253501.0, 0.00569), (1602.0, 0.00536), (4674.0, 0.00535), (7750.0, 0.00535), (8264.0, 0.00536), (7241.0, 0.00535), (65609.0, 0.00534), (23627.0, 0.00535), (5709.0, 0.00535), (183377.0, 0.00535), (9299.0, 0.00535), (152149.0, 0.00536), (14934.0, 0.00536), (12892.0, 0.00535), (146017.0, 0.00536), (110179.0, 0.00535), (65640.0, 0.00535), (34410.0, 0.00536), (13421.0, 0.00535), (186989.0, 0.00536), (5234.0, 0.00535), (10867.0, 0.00536), (64630.0, 0.00536), (147579.0, 0.00536), (61567.0, 0.00536), (4737.0, 0.00535), (6788.0, 0.00535), (5256.0, 0.00535), (133768.0, 0.00535), (287882.0,

cleaned_weights_min:  OrderedDict([(4611.0, 0.00546), (9225.0, 0.00541), (2574.0, 0.00578), (1045.0, 0.00532), (8215.0, 0.00527), (6682.0, 0.00541), (28192.0, 0.00523), (3107.0, 0.00506), (165927.0, 0.00513), (10793.0, 0.00547), (10795.0, 0.00554), (26156.0, 0.00531), (61483.0, 0.00522), (2101.0, 0.00538), (15417.0, 0.00501), (29241.0, 0.00532), (187450.0, 0.00525), (253501.0, 0.05), (162887.0, 0.00548), (8264.0, 0.0055), (7241.0, 0.00518), (65609.0, 0.00602), (23627.0, 0.00538), (5709.0, 0.00516), (6733.0, 0.00553), (27215.0, 0.00523), (183377.0, 0.00537), (9299.0, 0.00513), (152149.0, 0.00529), (6742.0, 0.00821), (14934.0, 0.00554), (7260.0, 0.00593), (12892.0, 0.00516), (146017.0, 0.00571), (110179.0, 0.00541), (65640.0, 0.00523), (8810.0, 0.00528), (34410.0, 0.00539), (29804.0, 0.00521), (13421.0, 0.00521), (186989.0, 0.0057), (4093.0, 0.00522), (5234.0, 0.00506), (28790.0, 0.00523), (18043.0, 0.00531), (6781.0, 0.00531), (61567.0, 0.00533), (61574.0, 0.00527), (178310.0, 0.00526),

cleaned_weights_min:  OrderedDict([(4611.0, 0.00538), (2574.0, 0.00538), (66065.0, 0.00538), (1045.0, 0.00538), (6682.0, 0.00538), (31774.0, 0.00538), (4640.0, 0.00538), (28192.0, 0.00538), (165927.0, 0.00535), (24616.0, 0.00538), (10793.0, 0.00538), (10795.0, 0.00538), (26156.0, 0.00537), (61483.0, 0.00538), (5680.0, 0.00538), (3121.0, 0.00538), (1075.0, 0.00538), (9783.0, 0.00538), (4674.0, 0.00538), (7750.0, 0.00538), (162887.0, 0.00538), (6216.0, 0.00538), (7241.0, 0.00538), (23627.0, 0.00539), (6733.0, 0.00538), (11856.0, 0.00538), (183377.0, 0.00538), (14418.0, 0.00538), (9299.0, 0.00538), (152149.0, 0.00538), (14934.0, 0.00535), (7260.0, 0.00538), (12892.0, 0.00538), (36444.0, 0.00538), (146017.0, 0.00538), (110179.0, 0.00541), (8810.0, 0.00538), (163946.0, 0.0054), (186989.0, 0.00538), (5742.0, 0.00538), (5234.0, 0.00538), (9846.0, 0.00538), (28790.0, 0.00538), (4737.0, 0.00538), (29830.0, 0.00538), (61574.0, 0.00537), (133768.0, 0.00538), (178310.0, 0.00538), (35978.0, 0.00538

cleaned_weights_min:  OrderedDict([(4108.0, 0.00565), (6669.0, 0.00565), (2574.0, 0.00565), (66065.0, 0.00565), (6682.0, 0.00565), (25119.0, 0.00565), (4640.0, 0.00565), (28192.0, 0.00565), (3619.0, 0.00565), (11300.0, 0.00565), (165927.0, 0.00565), (24616.0, 0.00565), (10793.0, 0.00565), (61483.0, 0.00565), (9778.0, 0.00565), (15417.0, 0.00565), (187450.0, 0.00565), (20548.0, 0.00565), (7750.0, 0.00565), (28742.0, 0.00565), (6216.0, 0.00565), (160329.0, 0.00565), (6733.0, 0.00565), (183377.0, 0.00565), (152149.0, 0.00565), (6742.0, 0.00565), (14934.0, 0.00565), (7260.0, 0.00565), (12892.0, 0.00565), (36444.0, 0.00565), (146017.0, 0.00565), (36967.0, 0.00565), (142953.0, 0.00565), (8810.0, 0.00565), (29804.0, 0.00565), (11376.0, 0.00565), (5234.0, 0.00565), (10867.0, 0.00565), (6774.0, 0.00565), (8823.0, 0.00565), (9846.0, 0.00565), (18043.0, 0.00565), (294524.0, 0.00565), (61567.0, 0.00565), (4737.0, 0.00565), (24197.0, 0.00565), (178310.0, 0.00565), (2184.0, 0.00565), (133768.0, 0.00

cleaned_weights_min:  OrderedDict([(4611.0, 0.004), (117768.0, 0.004), (100873.0, 0.004), (2574.0, 0.004), (175955.0, 0.004), (31774.0, 0.004), (25119.0, 0.004), (3107.0, 0.004), (11811.0, 0.004), (10789.0, 0.004), (28195.0, 0.004), (121382.0, 0.004), (24616.0, 0.004), (61483.0, 0.004), (26156.0, 0.004), (9778.0, 0.004), (30259.0, 0.004), (2101.0, 0.004), (15414.0, 0.004), (19000.0, 0.004), (15417.0, 0.004), (29241.0, 0.004), (187450.0, 0.004), (3650.0, 0.004), (20548.0, 0.004), (7750.0, 0.004), (3144.0, 0.004), (8264.0, 0.004), (65609.0, 0.004), (23627.0, 0.004), (5709.0, 0.004), (23119.0, 0.004), (183377.0, 0.004), (9299.0, 0.004), (152149.0, 0.004), (6742.0, 0.004), (141913.0, 0.004), (12892.0, 0.004), (36444.0, 0.004), (1632.0, 0.004), (146017.0, 0.004), (12389.0, 0.004), (63080.0, 0.004), (142953.0, 0.004), (163946.0, 0.004), (165993.0, 0.004), (13421.0, 0.004), (186989.0, 0.004), (11376.0, 0.004), (14960.0, 0.004), (177267.0, 0.004), (9846.0, 0.004), (28790.0, 0.004), (64630.0, 0

cleaned_weights_min:  OrderedDict([(11264.0, 0.00406), (7171.0, 0.00406), (17928.0, 0.00406), (100873.0, 0.00407), (117768.0, 0.00405), (2574.0, 0.00407), (1045.0, 0.00407), (36885.0, 0.00407), (136725.0, 0.00413), (31774.0, 0.00406), (28191.0, 0.00407), (3107.0, 0.00406), (2086.0, 0.00408), (121382.0, 0.00407), (24616.0, 0.00409), (10793.0, 0.00407), (61483.0, 0.00408), (36911.0, 0.00407), (4145.0, 0.00407), (9778.0, 0.00407), (1075.0, 0.00407), (12850.0, 0.00406), (2101.0, 0.00407), (1078.0, 0.00404), (15414.0, 0.00407), (19000.0, 0.00407), (15417.0, 0.00406), (61494.0, 0.00407), (161844.0, 0.00407), (7228.0, 0.00407), (187450.0, 0.00406), (2111.0, 0.00406), (1602.0, 0.00406), (7750.0, 0.00406), (162887.0, 0.00407), (141384.0, 0.00407), (7241.0, 0.00409), (6730.0, 0.00407), (23627.0, 0.00405), (22604.0, 0.00407), (5709.0, 0.00406), (6733.0, 0.00415), (23119.0, 0.00404), (65609.0, 0.00407), (160329.0, 0.00407), (183377.0, 0.00406), (9299.0, 0.00406), (152149.0, 0.00407), (6742.0, 0.00

cleaned_weights_min:  OrderedDict([(7171.0, 0.00292), (17928.0, 0.0032), (100873.0, 0.00289), (117768.0, 0.00274), (28180.0, 0.00249), (1045.0, 0.00299), (36885.0, 0.00317), (136725.0, 0.00299), (6682.0, 0.00294), (31774.0, 0.00296), (11811.0, 0.00283), (28195.0, 0.00302), (10789.0, 0.00301), (2086.0, 0.00303), (29733.0, 0.00292), (24616.0, 0.00312), (10793.0, 0.003), (121382.0, 0.00304), (10795.0, 0.00315), (26156.0, 0.00298), (61483.0, 0.00311), (165927.0, 0.00307), (36911.0, 0.00286), (4145.0, 0.00309), (9778.0, 0.00288), (1075.0, 0.00291), (1078.0, 0.00307), (9783.0, 0.00294), (15414.0, 0.00301), (15417.0, 0.00296), (19000.0, 0.00312), (29241.0, 0.00306), (7228.0, 0.00307), (61494.0, 0.00296), (187450.0, 0.003), (2111.0, 0.00296), (1602.0, 0.00305), (7750.0, 0.00287), (162887.0, 0.00282), (8264.0, 0.00296), (7241.0, 0.00298), (6730.0, 0.00357), (23627.0, 0.003), (22604.0, 0.00317), (5709.0, 0.0029), (6733.0, 0.00295), (23119.0, 0.00322), (141384.0, 0.00322), (160329.0, 0.00322), (1

In [33]:
stocks_weight_table.shape

(3870, 6)

In [34]:
stocks_weight_table.head()

,gvkey,predicted_return,trade_date,mean_weight,min_weight,equal_weight
0,1230,0.026920,2018-03-01,0.00590,0.00531,0.005556
1,1678,0.013570,2018-03-01,0.00572,0.00531,0.005556
2,1722,0.035988,2018-03-01,0.00572,0.00532,0.005556
3,2574,0.035439,2018-03-01,0.00549,0.00531,0.005556
4,2751,0.028170,2018-03-01,0.00572,0.00531,0.005556


## save to excel or csv

In [35]:
df_mean = pd.DataFrame()
df_mean['trade_date'] = stocks_weight_table['trade_date']
df_mean['gvkey'] = stocks_weight_table['gvkey']
df_mean['weights'] = stocks_weight_table['mean_weight']
df_mean['predicted_return'] = stocks_weight_table['predicted_return']
df_mean.to_excel("mean_weighted.xlsx")

In [36]:
df_min = pd.DataFrame()
df_min['trade_date'] = stocks_weight_table['trade_date']
df_min['gvkey'] = stocks_weight_table['gvkey']
df_min['weights'] = stocks_weight_table['min_weight']
df_min['predicted_return'] = stocks_weight_table['predicted_return']
df_min.to_excel("minimum_weighted.xlsx")

In [37]:
df_equal = pd.DataFrame()
df_equal['trade_date'] = stocks_weight_table['trade_date']
df_equal['gvkey'] = stocks_weight_table['gvkey']
df_equal['weights'] = stocks_weight_table['equal_weight']
df_equal['predicted_return'] = stocks_weight_table['predicted_return']
df_equal.to_excel("equally_weighted.xlsx")